In [1]:
import pandas as pd
from src.core import *
from typing import Optional
from dataclasses import dataclass
import torch
from torchvision import transforms
from selectivesearch import selective_search

**Task:**
Given region proposals for an image, annotate each Region of Interest (ROI) with
* Class that it belongs to
* Offset from the ground truth bounding box
* IOU of the ROI with the ground truth bounding box

In [2]:
df = pd.DataFrame(data['annotations'])
df.category_id = df.category_id.replace(cat_id2id)

In [3]:
tfms = transforms.Compose([
    transforms.ToTensor(),
])
ds = ObjectDataset(df, tfms)

In [4]:
@dataclass
class ROIs:
    boxes: torch.FloatTensor
    categories: Optional[torch.IntTensor] = None
    offsets: Optional[torch.FloatTensor] = None
    ious: Optional[torch.FloatTensor] = None
    
    def __post_init__(self):
        self.categories = torch.full((self.boxes.shape[0],), 0, dtype=torch.long)

In [5]:
def extract_rois(img):
    _, regions = selective_search(img.permute(1,2,0), scale=200, min_size=100)
    rois = torch.tensor([r['rect'] for r in regions])
    sizes = torch.tensor([r['size'] for r in regions])
    img_area = img.shape[1]*img.shape[2]
    mask = (sizes>0.05*img_area) & (sizes<img_area)
    return ROIs(rois[mask, :])

In [6]:
img, boxes, ids = ds[8]
rois = extract_rois(img)
print(rois.boxes.shape, boxes.shape)

/home/awesomeville/miniforge3/lib/python3.12/site-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


torch.Size([60, 4]) torch.Size([2, 4])


In [7]:
def get_ious(rois, bboxes, epsilon=1e-5):
    rois = torch.cat([rois[:,:2], rois[:,:2]+rois[:,2:]], dim=1)
    bboxes = torch.cat([bboxes[:,:2], bboxes[:,:2]+bboxes[:,2:]], dim=1)

    inter_xmin = torch.max(rois[:, None, 0], bboxes[None, :, 0])
    inter_ymin = torch.max(rois[:, None, 1], bboxes[None, :, 1])
    inter_xmax = torch.min(rois[:, None, 2], bboxes[None, :, 2])
    inter_ymax = torch.min(rois[:, None, 3], bboxes[None, :, 3])

    inter_w = (inter_xmax-inter_xmin).clamp(min=0)  # Ensure non-negative width
    inter_h = (inter_ymax-inter_ymin).clamp(min=0)  # Ensure non-negative height
    intersection = inter_w*inter_h
    
    area1 = (rois[:, 2]-rois[:, 0])*(rois[:, 3]-rois[:, 1])
    area2 = (bboxes[:, 2]-bboxes[:, 0])*(bboxes[:, 3]-bboxes[:, 1])
    union = area1[:, None]+area2[None, :]-intersection
    
    return intersection/(union+epsilon)

In [8]:
get_ious(rois.boxes, boxes).shape

torch.Size([60, 2])

In [10]:
def annotate_rois(bboxes, ids, rois, cat_thresh=0.3):
    ious = get_ious(rois.boxes, bboxes)
    max_ious, max_idxs = ious.max(dim=1)
    valid_mask = max_ious>cat_thresh
    rois.categories[valid_mask] = ids[max_idxs[valid_mask]]
    rois.offsets = bboxes[max_idxs]-rois.boxes
    rois.ious = max_ious
    return rois

In [12]:
rois = annotate_rois(boxes, ids, rois)